In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import math 

In [3]:
!git clone https://github.com/harvitronix/five-video-classification-methods

Cloning into 'five-video-classification-methods'...
remote: Enumerating objects: 495, done.
remote: Total 495 (delta 0), reused 0 (delta 0), pack-reused 495
Receiving objects: 100% (495/495), 322.37 KiB | 11.51 MiB/s, done.
Resolving deltas: 100% (289/289), done.


In [4]:
cd five-video-classification-methods/

/content/five-video-classification-methods


In [5]:
!pip install -r requirements.txt



In [6]:
!pip install ffmpeg

  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=e2f67737161c3e2cef71e0d697c49e913ac8895d318910d07784ccf78085ae6a
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg


In [7]:
cd data

/content/five-video-classification-methods/data


Downloading the full UCF-101 dataset in the data directory. We will be training on this dataset.



In [8]:
!wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101.rar

--2021-11-06 03:35:38--  https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 6932971618 (6.5G) [application/rar]
Saving to: ‘UCF101.rar’

UCF101.rar          100%[===================>]   6.46G  41.8MB/s    in 2m 38s  

2021-11-06 03:38:16 (41.8 MB/s) - ‘UCF101.rar’ saved [6932971618/6932971618]



In [14]:
!unrar e UCF101.rar

Streaming output truncated to the last 5000 lines.
Extracting  v_PlayingFlute_g25_c03.avi                                    61%  OK 
Extracting  v_PlayingFlute_g25_c04.avi                                    61%  OK 
Extracting  v_PlayingFlute_g25_c05.avi                                    61%  OK 
Extracting  v_PlayingFlute_g25_c06.avi                                    61%  OK 
Extracting  v_PlayingFlute_g25_c07.avi                                    61%  OK 
Extracting  v_PlayingGuitar_g01_c01.avi                                   61%  OK 
Extracting  v_PlayingGuitar_g01_c02.avi                                   61%  OK 
Extracting  v_PlayingGuitar_g01_c03.avi                                   61%  OK 
Extracting  v_PlayingGuitar_g01_c04.avi                                   61%  OK 
Extracting  v_PlayingGuitar_g01_c05.avi                                   61%  OK 
Extracting  v_PlayingGuitar_g

Creating training and test dataset. Checkpoints directory will store training progress

In [10]:
mkdir train && mkdir test && mkdir sequences && mkdir checkpoints

This code will split the videos to training and testing folder

In [13]:
pwd

'/content/five-video-classification-methods/data'

In [15]:
!python 1_move_files.py

Streaming output truncated to the last 5000 lines.
Moving v_Surfing_g19_c04.avi to train/Surfing/v_Surfing_g19_c04.avi
Moving v_Surfing_g20_c01.avi to train/Surfing/v_Surfing_g20_c01.avi
Moving v_Surfing_g20_c02.avi to train/Surfing/v_Surfing_g20_c02.avi
Moving v_Surfing_g20_c03.avi to train/Surfing/v_Surfing_g20_c03.avi
Moving v_Surfing_g20_c04.avi to train/Surfing/v_Surfing_g20_c04.avi
Moving v_Surfing_g20_c05.avi to train/Surfing/v_Surfing_g20_c05.avi
Moving v_Surfing_g20_c06.avi to train/Surfing/v_Surfing_g20_c06.avi
Moving v_Surfing_g21_c01.avi to train/Surfing/v_Surfing_g21_c01.avi
Moving v_Surfing_g21_c02.avi to train/Surfing/v_Surfing_g21_c02.avi
Moving v_Surfing_g21_c03.avi to train/Surfing/v_Surfing_g21_c03.avi
Moving v_Surfing_g21_c04.avi to train/Surfing/v_Surfing_g21_c04.avi
Moving v_Surfing_g22_c01.avi to train/Surfing/v_Surfing_g22_c01.avi
Moving v_Surfing_g22_c02.avi to train/Surfing/v_Surfing_g22_c02.avi
Moving v_Surfing_g22_c03.avi to train/Surfing/v_Surfing_g22_c03.a

In [16]:
!python 2_extract_files.py



Streaming output truncated to the last 5000 lines.
  libswscale      4.  8.100 /  4.  8.100
  libswresample   2.  9.100 /  2.  9.100
  libpostproc    54.  7.100 / 54.  7.100
Input #0, avi, from 'test/PlayingGuitar/v_PlayingGuitar_g07_c07.avi':
  Metadata:
    encoder         : Lavf52.31.1
  Duration: 00:00:10.00, start: 0.000000, bitrate: 514 kb/s
    Stream #0:0: Video: mpeg4 (Simple Profile) (xvid / 0x64697678), yuv420p, 320x240 [SAR 1:1 DAR 4:3], 507 kb/s, 25 fps, 25 tbr, 25 tbn, 25 tbc
Stream mapping:
  Stream #0:0 -> #0:0 (mpeg4 (native) -> mjpeg (native))
Press [q] to stop, [?] for help
[swscaler @ 0x56090e0b6000] deprecated pixel format used, make sure you did set range correctly
Output #0, image2, to 'test/PlayingGuitar/v_PlayingGuitar_g07_c07-%04d.jpg':
  Metadata:
    encoder         : Lavf57.83.100
    Stream #0:0: Video: mjpeg, yuvj420p(pc), 320x240 [SAR 1:1 DAR 4:3], q=2-31, 200 kb/s, 25 fps, 25 tbn, 25 tbc
    Metadata:
      encoder         : Lavc57.107.100 mjpeg
    Sid

In [17]:
cd ..

/content/five-video-classification-methods


The model used below is based on 3D Convolutional Network, C3D. This is the structure of the model

3D Conv —> Max pooling —> 3D Conv —> Max pooling —> 3D Conv —> 3D Conv —> Max pooling —> 3D Conv —> 3D Conv —> Max pooling —> Flatten —> Dense —> Dropout —> Dense —> Dropout —> Dense 

It consists of six 3D Conv layers with ReLU Activation, last 3 layers are fully connected layers. Dropout is used to prevent overfitting. The output layer is softmax with 101 outputs.

It is trained for 100 epochs with batch size 32

Since the dataset it takes a long time to train and the accuracy achieved after x iterations is 10%. We believe that the accuracy will improve if trained for longer time.

The paper reference is here : https://arxiv.org/pdf/1412.0767.pdf


With thanks : https://github.com/harvitronix/five-video-classification-methods

In [ ]:
!python train.py

2021-11-06 05:23:10.480600: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-06 05:23:10.480663: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-06 05:23:10.495338: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs
2021-11-06 05:23:10.500171: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-11-06 05:23:10.500318: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
Loading Conv3D
2021-11-06 05:23:10.540566: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-06 05:23:10.552306: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so r

This code uses 3D CNN model for action recognition on UCF-101 dataset.

---



3D Conv —> 3D Conv —> Max pool —> Dropout —> 3D Conv —> 3D Conv —> Max pooling —> Dropout —> Flatten —> Dense —> Dropout —> Dense

It consists of four 3D Conv layers with ReLU Activation, last 2 layers are fully connected layers. Dropout is used to prevent overfitting. The last layer has softmax activation with 10 outputs corresponding to the 10 action classes.

It is trained for 50 epochs with mini-batch size of 32 for 10 action classes

This can achieve 96.9% accuracy on the training set and 92% accuracy on the test set. 

Since the no of action classes is small, good accuracy can be achieved even though the model is simpler and trained for lower epochs.

With thanks : https://github.com/kcct-fujimotolab/3DCNN

In [ ]:
!git clone https://github.com/kcct-fujimotolab/3DCNN

Cloning into '3DCNN'...
remote: Enumerating objects: 248, done.
remote: Total 248 (delta 0), reused 0 (delta 0), pack-reused 248
Receiving objects: 100% (248/248), 121.56 MiB | 29.28 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
mkdir UCF101

In [ ]:
cd UCF101/

/content/five-video-classification-methods/UCF101


In [ ]:
!unrar e ../data/UCF101.rar

Streaming output truncated to the last 5000 lines.
Extracting  v_PlayingFlute_g25_c03.avi                                    61%  OK 
Extracting  v_PlayingFlute_g25_c04.avi                                    61%  OK 
Extracting  v_PlayingFlute_g25_c05.avi                                    61%  OK 
Extracting  v_PlayingFlute_g25_c06.avi                                    61%  OK 
Extracting  v_PlayingFlute_g25_c07.avi                                    61%  OK 
Extracting  v_PlayingGuitar_g01_c01.avi                                   61%  OK 
Extracting  v_PlayingGuitar_g01_c02.avi                                   61%  OK 
Extracting  v_PlayingGuitar_g01_c03.avi                                   61%  OK 
Extracting  v_PlayingGuitar_g01_c04.avi                                   61%  OK 
Extracting  v_PlayingGuitar_g01_c05.avi                                   61%  OK 
Extracting  v_PlayingGuitar_g

In [ ]:
pwd

'/content/five-video-classification-methods/UCF101'

In [ ]:
cd ..

/content/five-video-classification-methods/3DCNN


In [ ]:
!python videoto3d.py --batch 32 --epoch 50 --videos UCF101/ --nclass 10 --output 3dcnnresult/ --color True --skip False --depth 15

In [ ]:
!python 3dcnn.py --batch 32 --epoch 50 --videos dataset/ --nclass 10 --output 3dcnnresult/ --color True --skip False --depth 15


X_shape:(1280, 32, 32, 15, 3)
Y_shape:(1280, 10)
2021-11-04 05:14:01.145030: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 05:14:01.155421: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 05:14:01.156072: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 05:14:01.157111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-04 05:14:01.157736: I tensorflow/stream_executor/cuda/cuda_gpu_